# Lezione 33 — Named Entity Recognition (NER)

## BLOCCO 4: Artificial Intelligence & NLP

---

## Obiettivi di Apprendimento

Al termine di questa lezione sarai in grado di:

1. **Definire** cosa sono le Named Entities e perché estrarle
2. **Classificare** le tipologie di entità standard (PER, ORG, LOC, DATE, ecc.)
3. **Implementare** un sistema NER rule-based con regex
4. **Comprendere** le architetture ML per NER (BIO tagging)
5. **Applicare** NER per estrarre informazioni strutturate da testo

---

## Prerequisiti

- Lezione 30: Rappresentazione del testo (tokenizzazione)
- Lezione 32: Sentiment Analysis (classificazione testo)
- Nozioni base di espressioni regolari (regex)

---

## Indice

1. Teoria — Cos'è il NER
2. Tipologie di Entità
3. Schema Mentale
4. Notebook Dimostrativo
5. Esercizi Svolti
6. Conclusione e Bignami

---

# 1. Teoria — Cos'è il Named Entity Recognition

## 1.1 Definizione Formale

Il **Named Entity Recognition (NER)** è il task di NLP che consiste nell'identificare e classificare porzioni di testo che rappresentano entità del mondo reale.

```
Input: "Mario Rossi lavora presso Microsoft a Milano dal 2020."

Output:
  "Mario Rossi"  → PERSONA (PER)
  "Microsoft"    → ORGANIZZAZIONE (ORG)
  "Milano"       → LUOGO (LOC)
  "2020"         → DATA (DATE)
```

## 1.2 Perché il NER è Importante

| Applicazione | Uso del NER | Valore Business |
|--------------|-------------|-----------------|
| Information Extraction | Estrarre fatti da testo | Knowledge base automatiche |
| Customer Service | Identificare prodotti/servizi menzionati | Routing automatico |
| Compliance | Rilevare PII (Personally Identifiable Info) | GDPR, privacy |
| Finance | Estrarre ticker, aziende, date | Trading automatico |
| Healthcare | Identificare farmaci, sintomi, malattie | Cartelle cliniche |

## 1.3 Il NER come Sequence Labeling

A differenza della classificazione (1 label per documento), il NER assegna **1 label per ogni token**:

```
Token:    [Mario] [Rossi] [lavora] [presso] [Microsoft] [a] [Milano]
Labels:   [B-PER] [I-PER] [O]      [O]      [B-ORG]     [O] [B-LOC]
```

Questo è chiamato **sequence labeling** o **token classification**.

---

## 1.4 Tipologie di Entità Standard

Le entità più comuni seguite negli standard (CoNLL, OntoNotes):

| Tipo | Sigla | Esempi |
|------|-------|--------|
| **Persona** | PER | Mario Rossi, Angela Merkel |
| **Organizzazione** | ORG | Microsoft, ONU, Ferrari |
| **Luogo** | LOC | Milano, Italia, Monte Bianco |
| **Geopolitico** | GPE | Italia (come stato), Roma (come comune) |
| **Data** | DATE | 25 dicembre, 2024, lunedì |
| **Tempo** | TIME | 14:30, mezzogiorno |
| **Denaro** | MONEY | €100, 1 milione di euro |
| **Percentuale** | PERCENT | 5%, dieci per cento |
| **Prodotto** | PRODUCT | iPhone, Fiat 500 |
| **Evento** | EVENT | Olimpiadi, G20 |

## 1.5 Schema BIO (Begin-Inside-Outside)

Il formato più usato per annotare NER:

| Tag | Significato |
|-----|-------------|
| **B-XXX** | Begin: primo token di un'entità di tipo XXX |
| **I-XXX** | Inside: token successivi della stessa entità |
| **O** | Outside: non fa parte di nessuna entità |

**Esempio:**

```
Testo:  "La Ferrari ha vinto a Monza"
        La    Ferrari   ha   vinto   a    Monza
Tags:   O     B-ORG     O    O       O    B-LOC

Testo:  "Il CEO di Amazon Web Services"
        Il   CEO   di   Amazon   Web      Services
Tags:   O    O     O    B-ORG    I-ORG    I-ORG
```

**Perché B e I separati?**
Per distinguere entità adiacenti:
```
"New York Times cita New York Post"
 B-ORG I-ORG I-ORG  O    B-ORG I-ORG I-ORG
```

---

## 1.6 Approcci al NER

### Approccio 1: Rule-Based (Pattern Matching)

Usa regex e regole per identificare entità:

```python
# Esempio: date in formato italiano
import re
pattern_data = r'\d{1,2}[/-]\d{1,2}[/-]\d{2,4}'
# Match: "25/12/2024", "1-1-99"

# Esempio: email
pattern_email = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'
```

**Pro:** Precisione alta per pattern noti, interpretabile
**Contro:** Scarsa recall su varianti, non generalizza

### Approccio 2: Machine Learning (Feature-based)

Features estratte per ogni token:
- Il token stesso
- Prefisso/Suffisso
- È maiuscolo?
- È un numero?
- POS tag (verbo, sostantivo...)
- Token precedente/successivo

Classificatori: CRF (Conditional Random Fields), HMM, SVM

### Approccio 3: Deep Learning

- LSTM bidirezionali
- Transformers (BERT, RoBERTa)
- Fine-tuning su dati annotati

**Pro:** State-of-the-art performance
**Contro:** Richiede GPU, dati, competenze

## 1.7 Metriche per NER

**Entity-level** (la più usata):
- Un'entità è corretta se: tipo E span sono entrambi corretti

$$Precision = \frac{\text{Entità predette correttamente}}{\text{Tutte le entità predette}}$$

$$Recall = \frac{\text{Entità predette correttamente}}{\text{Tutte le entità reali}}$$

$$F1 = 2 \times \frac{Precision \times Recall}{Precision + Recall}$$

---

# 2. Schema Mentale — Mappa Logica

## Pipeline NER

```
                        NER PIPELINE
                        
┌─────────────────────────────────────────────────────────────┐
│                      INPUT: Testo Raw                       │
│     "Il CEO di Apple, Tim Cook, visiterà Roma il 15/3"     │
└─────────────────────────────────────────────────────────────┘
                              │
                              ▼
┌─────────────────────────────────────────────────────────────┐
│                    TOKENIZZAZIONE                           │
│   ["Il", "CEO", "di", "Apple", ",", "Tim", "Cook", ",",    │
│    "visiterà", "Roma", "il", "15/3"]                       │
└─────────────────────────────────────────────────────────────┘
                              │
                              ▼
┌─────────────────────────────────────────────────────────────┐
│                  FEATURE EXTRACTION                         │
│                                                             │
│   Token  │ IsUpper │ IsDigit │ Prefix │ Suffix │ Shape    │
│   ───────┼─────────┼─────────┼────────┼────────┼────────  │
│   Apple  │  True   │  False  │  App   │  ple   │  Xxxxx   │
│   Tim    │  True   │  False  │  Tim   │  im    │  Xxx     │
│   15/3   │  False  │  True   │  15/   │  /3    │  dd/d    │
└─────────────────────────────────────────────────────────────┘
                              │
                              ▼
┌─────────────────────────────────────────────────────────────┐
│                    CLASSIFICAZIONE                          │
│                                                             │
│   Rule-Based          ML (CRF)           Deep Learning     │
│   (regex, lookup)     (features)         (BERT, LSTM)      │
│                                                             │
│   → Per casi noti     → Bilanciato       → Massima acc.    │
│   → Interpretabile    → Moderate res.    → Richiede GPU    │
└─────────────────────────────────────────────────────────────┘
                              │
                              ▼
┌─────────────────────────────────────────────────────────────┐
│                     SPAN EXTRACTION                         │
│                                                             │
│   Token    │  BIO Tag   │  Entità Estratta                 │
│   ─────────┼────────────┼──────────────────                │
│   Apple    │  B-ORG     │  "Apple" (ORG)                   │
│   Tim      │  B-PER     │  "Tim Cook" (PER)                │
│   Cook     │  I-PER     │                                  │
│   Roma     │  B-LOC     │  "Roma" (LOC)                    │
│   15/3     │  B-DATE    │  "15/3" (DATE)                   │
└─────────────────────────────────────────────────────────────┘
                              │
                              ▼
┌─────────────────────────────────────────────────────────────┐
│                     OUTPUT STRUTTURATO                      │
│                                                             │
│   [                                                         │
│     {"text": "Apple", "type": "ORG", "start": 11, "end": 16}│
│     {"text": "Tim Cook", "type": "PER", "start": 18, "end": 26}
│     {"text": "Roma", "type": "LOC", "start": 37, "end": 41} │
│     {"text": "15/3", "type": "DATE", "start": 46, "end": 50}│
│   ]                                                         │
└─────────────────────────────────────────────────────────────┘
```

## Scelta dell'Approccio

```
                    QUALE APPROCCIO?
                    
              Entità con pattern fisso?
              (email, date, codici)
                     │
         ┌───────────┴───────────┐
         │ Sì                    │ No
         ▼                       ▼
    REGEX/RULES            Dati annotati?
                                 │
                     ┌───────────┴───────────┐
                     │ Sì                    │ No
                     ▼                       ▼
               Quanti dati?           Usa modello
                     │                pre-trained
         ┌───────────┴───────────┐    (spaCy, HF)
         │ <5000              │ >50000
         ▼                       ▼
       CRF                 Fine-tune BERT
```

---

# 3. Notebook Dimostrativo

## Setup e Utilities

In [ ]:
# ============================================================
# DEMO 1: NER Rule-Based con Regex
# ============================================================

import re
from typing import List, Dict, Tuple
from collections import namedtuple

# Definiamo una struttura per le entità estratte
Entity = namedtuple('Entity', ['text', 'type', 'start', 'end'])

class RuleBasedNER:
    """
    Sistema NER basato su regole (regex).
    Ideale per entità con pattern prevedibili.
    """
    
    def __init__(self):
        # Dizionario di pattern regex per tipo di entità
        self.patterns = {
            # Date italiane: 25/12/2024, 25-12-24, 25 dicembre 2024
            'DATE': [
                r'\b\d{1,2}[/-]\d{1,2}[/-]\d{2,4}\b',
                r'\b\d{1,2}\s+(gennaio|febbraio|marzo|aprile|maggio|giugno|'
                r'luglio|agosto|settembre|ottobre|novembre|dicembre)\s+\d{4}\b',
            ],
            
            # Email
            'EMAIL': [
                r'\b[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}\b',
            ],
            
            # Telefoni italiani
            'PHONE': [
                r'\b(?:\+39\s?)?(?:0\d{1,4}|\d{3})[\s.-]?\d{6,7}\b',
                r'\b(?:\+39\s?)?3\d{2}[\s.-]?\d{6,7}\b',  # Cellulari
            ],
            
            # Codice fiscale italiano
            'CF': [
                r'\b[A-Z]{6}\d{2}[A-Z]\d{2}[A-Z]\d{3}[A-Z]\b',
            ],
            
            # Importi monetari
            'MONEY': [
                r'€\s?\d+(?:[.,]\d{2})?(?:\s?(?:mila|milioni|miliardi))?',
                r'\d+(?:[.,]\d{2})?\s?€',
                r'\d+(?:[.,]\d{2})?\s?euro\b',
            ],
            
            # Percentuali
            'PERCENT': [
                r'\d+(?:[.,]\d+)?\s?%',
                r'\d+(?:[.,]\d+)?\s?per\s?cento',
            ],
        }
    
    def extract(self, text: str) -> List[Entity]:
        """
        Estrae entità dal testo usando i pattern definiti.
        
        Args:
            text: testo da analizzare
            
        Returns:
            Lista di Entity trovate
        """
        entities = []
        
        for entity_type, patterns in self.patterns.items():
            for pattern in patterns:
                # Trova tutti i match
                for match in re.finditer(pattern, text, re.IGNORECASE):
                    entity = Entity(
                        text=match.group(),
                        type=entity_type,
                        start=match.start(),
                        end=match.end()
                    )
                    entities.append(entity)
        
        # Ordina per posizione
        entities.sort(key=lambda x: x.start)
        
        # Rimuovi overlap (mantieni il più lungo)
        entities = self._remove_overlaps(entities)
        
        return entities
    
    def _remove_overlaps(self, entities: List[Entity]) -> List[Entity]:
        """Rimuove entità sovrapposte, mantenendo la più lunga."""
        if not entities:
            return []
        
        result = [entities[0]]
        for entity in entities[1:]:
            last = result[-1]
            # Se non c'è overlap, aggiungi
            if entity.start >= last.end:
                result.append(entity)
            # Se c'è overlap, tieni il più lungo
            elif len(entity.text) > len(last.text):
                result[-1] = entity
        
        return result

# Test del sistema
ner = RuleBasedNER()

testi_test = [
    "Contattami al 333-1234567 o via email mario.rossi@gmail.com",
    "Il fatturato è cresciuto del 15% raggiungendo €2,5 milioni",
    "La riunione è fissata per il 25/12/2024 alle 15:30",
    "Codice fiscale: RSSMRA80A01H501X",
]

print("="*60)
print("DEMO 1: NER RULE-BASED")
print("="*60)

for testo in testi_test:
    print(f"\nTesto: '{testo}'")
    entities = ner.extract(testo)
    if entities:
        print("  Entità trovate:")
        for e in entities:
            print(f"    [{e.type}] '{e.text}' (pos: {e.start}-{e.end})")
    else:
        print("  Nessuna entità trovata")

In [ ]:
# ============================================================
# DEMO 2: NER con Lookup Tables (Gazetteers)
# ============================================================
# Per nomi propri e organizzazioni, usiamo liste predefinite

class GazetteerNER:
    """
    NER basato su dizionari (gazetteers).
    Efficace per entità note a priori.
    """
    
    def __init__(self):
        # Gazetteers: liste di entità note
        self.gazetteers = {
            'PER': [
                # Nomi italiani comuni
                'mario', 'luigi', 'giuseppe', 'giovanni', 'antonio',
                'francesco', 'alessandro', 'andrea', 'marco', 'luca',
                'maria', 'anna', 'giulia', 'francesca', 'sara',
                # Cognomi comuni
                'rossi', 'russo', 'ferrari', 'esposito', 'bianchi',
                'romano', 'colombo', 'ricci', 'marino', 'greco',
            ],
            'ORG': [
                'apple', 'microsoft', 'google', 'amazon', 'facebook', 'meta',
                'ferrari', 'fiat', 'eni', 'enel', 'unicredit', 'intesa',
                'juventus', 'inter', 'milan', 'roma', 'napoli',
                'onu', 'nato', 'ue', 'unione europea', 'bce',
            ],
            'LOC': [
                'roma', 'milano', 'napoli', 'torino', 'firenze', 'bologna',
                'venezia', 'genova', 'palermo', 'bari', 'catania',
                'italia', 'francia', 'germania', 'spagna', 'usa',
                'europa', 'asia', 'africa', 'america',
            ],
        }
        
        # Crea lookup set per ricerca veloce (lowercase)
        self.lookup = {
            entity_type: set(name.lower() for name in names)
            for entity_type, names in self.gazetteers.items()
        }
    
    def extract(self, text: str) -> List[Entity]:
        """Estrae entità usando i gazetteers."""
        entities = []
        words = text.split()
        
        # Cerca singole parole
        for i, word in enumerate(words):
            word_clean = re.sub(r'[^\w]', '', word.lower())
            
            for entity_type, lookup_set in self.lookup.items():
                if word_clean in lookup_set:
                    # Trova posizione nel testo originale
                    start = text.lower().find(word_clean)
                    if start != -1:
                        entities.append(Entity(
                            text=word.strip('.,!?'),
                            type=entity_type,
                            start=start,
                            end=start + len(word_clean)
                        ))
        
        # Cerca multi-word (bigram)
        for i in range(len(words) - 1):
            bigram = f"{words[i]} {words[i+1]}".lower()
            bigram_clean = re.sub(r'[^\w\s]', '', bigram)
            
            for entity_type, lookup_set in self.lookup.items():
                if bigram_clean in lookup_set:
                    start = text.lower().find(bigram_clean)
                    if start != -1:
                        entities.append(Entity(
                            text=f"{words[i]} {words[i+1]}".strip('.,!?'),
                            type=entity_type,
                            start=start,
                            end=start + len(bigram_clean)
                        ))
        
        return sorted(set(entities), key=lambda x: x.start)

# Test
gazetteer_ner = GazetteerNER()

testi = [
    "Mario Rossi lavora a Milano per Unicredit",
    "La Juventus ha battuto il Milan a Torino",
    "Apple e Microsoft dominano il mercato tech",
]

print("="*60)
print("DEMO 2: NER CON GAZETTEERS")
print("="*60)

for testo in testi:
    print(f"\nTesto: '{testo}'")
    entities = gazetteer_ner.extract(testo)
    if entities:
        print("  Entità trovate:")
        for e in entities:
            print(f"    [{e.type}] '{e.text}'")

In [ ]:
# ============================================================
# DEMO 3: Formato BIO e Conversione
# ============================================================
# Comprendere il formato BIO usato nell'annotazione NER

def text_to_bio(text: str, entities: List[Tuple[int, int, str]]) -> List[Tuple[str, str]]:
    """
    Converte testo con entità annotate in formato BIO.
    
    Args:
        text: testo originale
        entities: lista di (start, end, type)
        
    Returns:
        Lista di (token, bio_tag)
    """
    # Tokenizzazione semplice
    tokens = []
    current_pos = 0
    
    for match in re.finditer(r'\S+', text):
        tokens.append({
            'text': match.group(),
            'start': match.start(),
            'end': match.end()
        })
    
    # Assegna tag BIO
    bio_tags = []
    for token in tokens:
        tag = 'O'  # Default: Outside
        
        for ent_start, ent_end, ent_type in entities:
            # Token inizia dentro l'entità
            if ent_start <= token['start'] < ent_end:
                # È il primo token dell'entità?
                if token['start'] == ent_start:
                    tag = f'B-{ent_type}'
                else:
                    tag = f'I-{ent_type}'
                break
        
        bio_tags.append((token['text'], tag))
    
    return bio_tags

def bio_to_entities(bio_tags: List[Tuple[str, str]]) -> List[Dict]:
    """
    Converte sequenza BIO in entità strutturate.
    
    Args:
        bio_tags: lista di (token, tag)
        
    Returns:
        Lista di {text, type}
    """
    entities = []
    current_entity = None
    
    for token, tag in bio_tags:
        if tag.startswith('B-'):
            # Salva entità precedente se esiste
            if current_entity:
                entities.append(current_entity)
            # Inizia nuova entità
            current_entity = {
                'text': token,
                'type': tag[2:]  # Rimuove "B-"
            }
        elif tag.startswith('I-') and current_entity:
            # Continua entità corrente
            current_entity['text'] += ' ' + token
        else:
            # O tag: chiudi entità se aperta
            if current_entity:
                entities.append(current_entity)
                current_entity = None
    
    # Non dimenticare l'ultima
    if current_entity:
        entities.append(current_entity)
    
    return entities

# Esempio: testo → BIO
testo = "Mario Rossi lavora presso Microsoft a Milano"
# Annotazioni: (start, end, type)
annotations = [
    (0, 11, 'PER'),    # "Mario Rossi"
    (28, 37, 'ORG'),   # "Microsoft"
    (40, 46, 'LOC'),   # "Milano"
]

print("="*60)
print("DEMO 3: FORMATO BIO")
print("="*60)
print(f"\nTesto: '{testo}'")

# Converti in BIO
bio = text_to_bio(testo, annotations)
print(f"\nFormato BIO:")
print("-" * 30)
print(f"{'Token':<15} {'Tag':<10}")
print("-" * 30)
for token, tag in bio:
    print(f"{token:<15} {tag:<10}")

# Ricostruisci da BIO
print(f"\nRicostruzione da BIO:")
reconstructed = bio_to_entities(bio)
for ent in reconstructed:
    print(f"  [{ent['type']}] '{ent['text']}'")

In [ ]:
# ============================================================
# DEMO 4: Feature Engineering per NER ML-based
# ============================================================
# Quali feature estraiamo da ogni token per un modello ML

import numpy as np

def extract_token_features(token: str, prev_token: str = None, 
                           next_token: str = None) -> Dict:
    """
    Estrae feature da un singolo token per NER.
    
    Args:
        token: il token corrente
        prev_token: token precedente (contesto)
        next_token: token successivo (contesto)
        
    Returns:
        Dizionario di feature
    """
    features = {
        # Feature sul token stesso
        'token': token.lower(),
        'len': len(token),
        
        # Caratteristiche morfologiche
        'is_upper': token.isupper(),            # "NATO"
        'is_title': token.istitle(),            # "Mario"
        'is_lower': token.islower(),            # "ciao"
        'is_digit': token.isdigit(),            # "2024"
        'is_alpha': token.isalpha(),            # "ciao"
        'is_alnum': token.isalnum(),            # "test123"
        
        # Prefissi e suffissi (utili per nomi/verbi)
        'prefix_2': token[:2].lower() if len(token) >= 2 else token.lower(),
        'prefix_3': token[:3].lower() if len(token) >= 3 else token.lower(),
        'suffix_2': token[-2:].lower() if len(token) >= 2 else token.lower(),
        'suffix_3': token[-3:].lower() if len(token) >= 3 else token.lower(),
        
        # Shape: pattern del token (X=maiuscola, x=minuscola, d=digit)
        'shape': get_token_shape(token),
        
        # Contiene caratteri speciali?
        'has_hyphen': '-' in token,
        'has_digit': any(c.isdigit() for c in token),
        'has_punct': any(not c.isalnum() for c in token),
    }
    
    # Feature di contesto
    if prev_token:
        features['prev_token'] = prev_token.lower()
        features['prev_is_title'] = prev_token.istitle()
    else:
        features['prev_token'] = '<START>'
        features['prev_is_title'] = False
    
    if next_token:
        features['next_token'] = next_token.lower()
        features['next_is_title'] = next_token.istitle()
    else:
        features['next_token'] = '<END>'
        features['next_is_title'] = False
    
    return features

def get_token_shape(token: str) -> str:
    """
    Genera la 'shape' di un token.
    Esempio: "McDonald's" → "XxXxxxx'x"
    """
    shape = []
    for c in token:
        if c.isupper():
            shape.append('X')
        elif c.islower():
            shape.append('x')
        elif c.isdigit():
            shape.append('d')
        else:
            shape.append(c)
    
    # Comprimi sequenze ripetute
    if len(shape) > 0:
        compressed = [shape[0]]
        for s in shape[1:]:
            if s != compressed[-1] or s not in 'Xxd':
                compressed.append(s)
        return ''.join(compressed)
    return ''

# Dimostra su alcuni token
tokens_demo = ["Mario", "ROSSI", "lavora", "25/12/2024", "Microsoft", "€100"]

print("="*60)
print("DEMO 4: FEATURE ENGINEERING")
print("="*60)

for i, token in enumerate(tokens_demo):
    prev_t = tokens_demo[i-1] if i > 0 else None
    next_t = tokens_demo[i+1] if i < len(tokens_demo)-1 else None
    
    features = extract_token_features(token, prev_t, next_t)
    
    print(f"\nToken: '{token}'")
    print(f"  Shape: {features['shape']}")
    print(f"  is_title: {features['is_title']}, is_upper: {features['is_upper']}")
    print(f"  prefix_3: {features['prefix_3']}, suffix_3: {features['suffix_3']}")
    print(f"  Context: prev='{features['prev_token']}', next='{features['next_token']}'")

In [ ]:
# ============================================================
# DEMO 5: Sistema NER Ibrido (Regex + Gazetteer + Heuristics)
# ============================================================
# Combiniamo le tecniche per un sistema più robusto

class HybridNER:
    """
    Sistema NER che combina:
    1. Pattern regex per entità strutturate
    2. Gazetteers per nomi noti
    3. Heuristics per pattern linguistici
    """
    
    def __init__(self):
        self.rule_ner = RuleBasedNER()
        self.gazetteer_ner = GazetteerNER()
    
    def extract(self, text: str) -> List[Entity]:
        """Combina tutti gli approcci."""
        all_entities = []
        
        # 1. Entità da regex (date, email, money, etc.)
        regex_entities = self.rule_ner.extract(text)
        all_entities.extend(regex_entities)
        
        # 2. Entità da gazetteer (PER, ORG, LOC)
        gazetteer_entities = self.gazetteer_ner.extract(text)
        all_entities.extend(gazetteer_entities)
        
        # 3. Heuristics: pattern capitalized words non ancora catturati
        heuristic_entities = self._extract_heuristic(text, all_entities)
        all_entities.extend(heuristic_entities)
        
        # Rimuovi duplicati e overlap
        return self._resolve_conflicts(all_entities)
    
    def _extract_heuristic(self, text: str, existing: List[Entity]) -> List[Entity]:
        """
        Euristica: sequenze di parole con iniziale maiuscola
        potrebbero essere nomi propri non nel gazetteer.
        """
        entities = []
        existing_spans = {(e.start, e.end) for e in existing}
        
        # Pattern: 2+ parole consecutive che iniziano con maiuscola
        pattern = r'\b([A-Z][a-z]+(?:\s+[A-Z][a-z]+)+)\b'
        
        for match in re.finditer(pattern, text):
            # Salta se già catturato
            if (match.start(), match.end()) not in existing_spans:
                # Euristica: probabile nome proprio
                entities.append(Entity(
                    text=match.group(),
                    type='PER?',  # Incerto
                    start=match.start(),
                    end=match.end()
                ))
        
        return entities
    
    def _resolve_conflicts(self, entities: List[Entity]) -> List[Entity]:
        """Risolve conflitti tra entità sovrapposte."""
        if not entities:
            return []
        
        # Ordina per posizione
        sorted_ents = sorted(entities, key=lambda x: (x.start, -len(x.text)))
        
        result = []
        for ent in sorted_ents:
            # Controlla overlap con entità già accettate
            overlaps = False
            for accepted in result:
                if not (ent.end <= accepted.start or ent.start >= accepted.end):
                    overlaps = True
                    break
            
            if not overlaps:
                result.append(ent)
        
        return sorted(result, key=lambda x: x.start)

# Test del sistema ibrido
hybrid_ner = HybridNER()

testo_complesso = """
Il CEO di Apple, Tim Cook, ha annunciato che l'azienda raggiungerà
€10 miliardi di fatturato in Italia entro il 2025. 
La conferenza stampa si terrà a Milano il 25/03/2025.
Per informazioni: info@apple.com o 02-1234567.
"""

print("="*60)
print("DEMO 5: SISTEMA NER IBRIDO")
print("="*60)
print(f"\nTesto:\n{testo_complesso.strip()}")

entities = hybrid_ner.extract(testo_complesso)

print(f"\n{'='*60}")
print("ENTITÀ ESTRATTE")
print("="*60)
print(f"\n{'Tipo':<10} {'Testo':<30} {'Posizione':<15}")
print("-" * 55)
for e in entities:
    print(f"{e.type:<10} '{e.text}'".ljust(42) + f" ({e.start}-{e.end})")

In [ ]:
# ============================================================
# DEMO 6: Valutazione Metriche NER
# ============================================================
# Come calcolare Precision, Recall, F1 per NER

def evaluate_ner(predictions: List[Entity], 
                 ground_truth: List[Entity],
                 strict: bool = True) -> Dict:
    """
    Valuta le performance di un sistema NER.
    
    Args:
        predictions: entità predette
        ground_truth: entità corrette (gold standard)
        strict: se True, richiede match esatto di span E tipo
                se False, richiede solo overlap di span
    
    Returns:
        Dizionario con P, R, F1 per tipo
    """
    # Converti in set di tuple per confronto
    if strict:
        pred_set = {(e.text.lower(), e.type, e.start, e.end) for e in predictions}
        gold_set = {(e.text.lower(), e.type, e.start, e.end) for e in ground_truth}
    else:
        # Match rilassato: solo tipo deve coincidere
        pred_set = {(e.type, e.text.lower()) for e in predictions}
        gold_set = {(e.type, e.text.lower()) for e in ground_truth}
    
    # True Positives: predetti correttamente
    tp = len(pred_set & gold_set)
    
    # False Positives: predetti ma non nel gold
    fp = len(pred_set - gold_set)
    
    # False Negatives: nel gold ma non predetti
    fn = len(gold_set - pred_set)
    
    # Metriche
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    
    return {
        'TP': tp,
        'FP': fp,
        'FN': fn,
        'Precision': precision,
        'Recall': recall,
        'F1': f1
    }

# Esempio di valutazione
# Ground truth (annotazioni umane)
gold_entities = [
    Entity('Apple', 'ORG', 12, 17),
    Entity('Tim Cook', 'PER', 19, 27),
    Entity('€10 miliardi', 'MONEY', 72, 84),
    Entity('Italia', 'LOC', 103, 109),
    Entity('2025', 'DATE', 120, 124),
    Entity('Milano', 'LOC', 157, 163),
    Entity('25/03/2025', 'DATE', 167, 177),
    Entity('info@apple.com', 'EMAIL', 198, 212),
    Entity('02-1234567', 'PHONE', 215, 225),
]

# Predizioni del nostro sistema (dalla demo precedente)
pred_entities = entities  # Dal demo 5

print("="*60)
print("DEMO 6: VALUTAZIONE NER")
print("="*60)

# Valutazione stretta
strict_metrics = evaluate_ner(pred_entities, gold_entities, strict=True)
print("\n📊 Valutazione STRICT (span + tipo esatti):")
print(f"  True Positives: {strict_metrics['TP']}")
print(f"  False Positives: {strict_metrics['FP']}")
print(f"  False Negatives: {strict_metrics['FN']}")
print(f"  Precision: {strict_metrics['Precision']:.2%}")
print(f"  Recall: {strict_metrics['Recall']:.2%}")
print(f"  F1-Score: {strict_metrics['F1']:.2%}")

# Valutazione rilassata
relaxed_metrics = evaluate_ner(pred_entities, gold_entities, strict=False)
print(f"\n📊 Valutazione RELAXED (solo tipo corretto):")
print(f"  Precision: {relaxed_metrics['Precision']:.2%}")
print(f"  Recall: {relaxed_metrics['Recall']:.2%}")
print(f"  F1-Score: {relaxed_metrics['F1']:.2%}")

---

# 4. Esercizi Svolti

## Esercizio 1: Estrazione Dati Strutturati da Testo

**Obiettivo:** Dato un testo non strutturato, estrarre automaticamente informazioni strutturate in formato JSON.

In [ ]:
# ============================================================
# ESERCIZIO 1 - SOLUZIONE
# Estrazione Dati Strutturati da Testo
# ============================================================

import json

class BusinessCardExtractor:
    """
    Estrae informazioni strutturate da un biglietto da visita
    o testo contenente dati di contatto.
    """
    
    def __init__(self):
        # Pattern per vari tipi di informazioni
        self.patterns = {
            'email': r'\b[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}\b',
            'telefono': r'(?:\+39\s?)?(?:0\d{1,4}|\d{3})[\s.-]?\d{6,7}',
            'cellulare': r'(?:\+39\s?)?3\d{2}[\s.-]?\d{6,7}',
            'piva': r'\b[0-9]{11}\b',
            'cf': r'\b[A-Z]{6}\d{2}[A-Z]\d{2}[A-Z]\d{3}[A-Z]\b',
            'cap': r'\b\d{5}\b',
            'indirizzo': r'(?:Via|Viale|Corso|Piazza|P\.zza)\s+[A-Za-zÀ-ÿ\s]+,?\s*\d+',
        }
        
        # Titoli che indicano ruoli
        self.titoli = [
            'CEO', 'CTO', 'CFO', 'COO', 'CMO',
            'Direttore', 'Manager', 'Responsabile',
            'Ingegnere', 'Ing.', 'Dott.', 'Dr.',
            'Avvocato', 'Avv.', 'Architetto', 'Arch.'
        ]
    
    def extract(self, testo: str) -> Dict:
        """Estrae tutte le informazioni disponibili."""
        risultato = {
            'nome': None,
            'ruolo': None,
            'azienda': None,
            'email': [],
            'telefono': [],
            'cellulare': [],
            'indirizzo': None,
            'piva': None,
            'cf': None,
        }
        
        # Estrai con regex
        for campo, pattern in self.patterns.items():
            matches = re.findall(pattern, testo, re.IGNORECASE)
            if matches:
                if campo in ['email', 'telefono', 'cellulare']:
                    risultato[campo] = matches
                else:
                    risultato[campo] = matches[0]
        
        # Estrai ruolo (cerca titoli)
        for titolo in self.titoli:
            if titolo.lower() in testo.lower():
                # Cerca contesto attorno al titolo
                pattern = rf'{titolo}\s*[:\-]?\s*\w+(?:\s+\w+)?'
                match = re.search(pattern, testo, re.IGNORECASE)
                if match:
                    risultato['ruolo'] = match.group().strip()
                    break
        
        # Estrai nome: prima riga o sequenza titolata
        righe = testo.strip().split('\n')
        if righe:
            prima_riga = righe[0].strip()
            # Se è corta e titlecase, probabilmente è il nome
            if len(prima_riga) < 50 and prima_riga.istitle():
                risultato['nome'] = prima_riga
        
        return risultato

# Test
extractor = BusinessCardExtractor()

biglietti = [
    """
    Marco Bianchi
    CEO - TechCorp Italia
    Via Roma 123, 20100 Milano
    Tel: 02-12345678
    Cell: 333-1234567
    Email: m.bianchi@techcorp.it
    P.IVA: 12345678901
    """,
    
    """
    Dott.ssa Giulia Verdi
    Responsabile Marketing
    Innovare S.r.l.
    Corso Vittorio Emanuele 45
    10121 Torino
    giulia.verdi@innovare.com
    +39 011 9876543
    """,
]

print("="*60)
print("ESERCIZIO 1: ESTRAZIONE DATI STRUTTURATI")
print("="*60)

for i, biglietto in enumerate(biglietti, 1):
    print(f"\n📇 Biglietto {i}:")
    print("-" * 40)
    
    dati = extractor.extract(biglietto)
    
    # Output formattato
    for chiave, valore in dati.items():
        if valore:
            print(f"  {chiave.capitalize()}: {valore}")
    
    # JSON output
    print(f"\n  📋 JSON:")
    print(f"  {json.dumps(dati, indent=4, ensure_ascii=False)}")

---

## Esercizio 2: PII Detection (Privacy)

**Obiettivo:** Creare un sistema che identifica e maschera dati personali sensibili (PII - Personally Identifiable Information) per conformità GDPR.

In [ ]:
# ============================================================
# ESERCIZIO 2 - SOLUZIONE
# PII Detection e Masking per GDPR
# ============================================================

class PIIDetector:
    """
    Sistema per identificare e mascherare dati personali
    secondo le normative sulla privacy (GDPR).
    
    PII Types:
    - Nomi e cognomi
    - Email
    - Numeri di telefono
    - Codici fiscali
    - IBAN
    - Carte di credito
    - Indirizzi IP
    - Date di nascita
    """
    
    def __init__(self):
        self.patterns = {
            'EMAIL': {
                'regex': r'\b[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}\b',
                'mask': '[EMAIL]'
            },
            'TELEFONO': {
                'regex': r'(?:\+39\s?)?(?:0\d{1,4}|\d{3})[\s.-]?\d{6,7}',
                'mask': '[TELEFONO]'
            },
            'CODICE_FISCALE': {
                'regex': r'\b[A-Z]{6}\d{2}[A-Z]\d{2}[A-Z]\d{3}[A-Z]\b',
                'mask': '[CODICE_FISCALE]'
            },
            'IBAN': {
                'regex': r'\b[A-Z]{2}\d{2}[A-Z]\d{22}\b|\bIT\d{2}[A-Z]\d{10}[A-Z0-9]{12}\b',
                'mask': '[IBAN]'
            },
            'CARTA_CREDITO': {
                'regex': r'\b(?:\d{4}[\s-]?){3}\d{4}\b',
                'mask': '[CARTA_CREDITO]'
            },
            'IP_ADDRESS': {
                'regex': r'\b(?:\d{1,3}\.){3}\d{1,3}\b',
                'mask': '[IP_ADDRESS]'
            },
            'DATA_NASCITA': {
                'regex': r'\b\d{1,2}[/-]\d{1,2}[/-](?:19|20)\d{2}\b',
                'mask': '[DATA_NASCITA]'
            },
        }
    
    def detect(self, testo: str) -> List[Dict]:
        """
        Identifica tutti i PII nel testo.
        
        Returns:
            Lista di {tipo, valore, posizione}
        """
        trovati = []
        
        for pii_type, config in self.patterns.items():
            for match in re.finditer(config['regex'], testo):
                trovati.append({
                    'tipo': pii_type,
                    'valore': match.group(),
                    'start': match.start(),
                    'end': match.end()
                })
        
        return sorted(trovati, key=lambda x: x['start'])
    
    def mask(self, testo: str) -> str:
        """
        Maschera tutti i PII trovati nel testo.
        
        Returns:
            Testo con PII mascherati
        """
        # Ordina i match dal fondo per non alterare le posizioni
        matches = self.detect(testo)
        matches_sorted = sorted(matches, key=lambda x: x['start'], reverse=True)
        
        masked = testo
        for match in matches_sorted:
            pii_type = match['tipo']
            mask_text = self.patterns[pii_type]['mask']
            masked = masked[:match['start']] + mask_text + masked[match['end']:]
        
        return masked
    
    def report(self, testo: str) -> Dict:
        """Genera un report sui PII trovati."""
        trovati = self.detect(testo)
        
        report = {
            'totale_pii': len(trovati),
            'per_tipo': {},
            'rischio': 'ALTO' if len(trovati) > 3 else 'MEDIO' if len(trovati) > 0 else 'BASSO'
        }
        
        for pii in trovati:
            tipo = pii['tipo']
            if tipo not in report['per_tipo']:
                report['per_tipo'][tipo] = []
            report['per_tipo'][tipo].append(pii['valore'])
        
        return report

# Test
detector = PIIDetector()

documento = """
Gentile Mario Rossi (CF: RSSMRA80A01H501X),

Confermiamo la sua prenotazione per il 25/12/1980 (data di nascita verificata).
Il suo ordine verrà spedito all'indirizzo registrato.

Dati di fatturazione:
- Email: mario.rossi@gmail.com
- Telefono: 333-1234567
- IBAN: IT60X0542811101000000123456
- Carta: 4532-1234-5678-9012

Accesso registrato da IP: 192.168.1.100

Cordiali saluti,
Il Team
"""

print("="*60)
print("ESERCIZIO 2: PII DETECTION")
print("="*60)

# Detect
print("\n📋 DOCUMENTO ORIGINALE:")
print("-" * 40)
print(documento)

# Report
report = detector.report(documento)
print(f"\n{'='*60}")
print("📊 REPORT PII")
print("="*60)
print(f"Totale PII trovati: {report['totale_pii']}")
print(f"Livello di rischio: {report['rischio']}")
print(f"\nPer tipo:")
for tipo, valori in report['per_tipo'].items():
    print(f"  {tipo}: {len(valori)} occorrenze")
    for v in valori:
        print(f"    - {v}")

# Masked version
masked = detector.mask(documento)
print(f"\n{'='*60}")
print("🔒 DOCUMENTO MASCHERATO (GDPR-compliant)")
print("="*60)
print(masked)

---

## Esercizio 3: News Analyzer con NER

**Obiettivo:** Analizzare un corpus di notizie estraendo automaticamente le entità più menzionate per tipo.

In [ ]:
# ============================================================
# ESERCIZIO 3 - SOLUZIONE
# News Analyzer con NER
# ============================================================

from collections import Counter
import pandas as pd

class NewsAnalyzer:
    """
    Analizza un corpus di notizie per estrarre:
    - Entità più menzionate
    - Co-occorrenze tra entità
    - Trend per data
    """
    
    def __init__(self):
        self.ner = HybridNER()  # Dal demo precedente
        self.corpus_entities = []
    
    def process_news(self, notizie: List[Dict]) -> None:
        """
        Processa un corpus di notizie.
        
        Args:
            notizie: Lista di {titolo, testo, data}
        """
        self.corpus_entities = []
        
        for news in notizie:
            # Combina titolo e testo
            full_text = f"{news.get('titolo', '')} {news.get('testo', '')}"
            
            # Estrai entità
            entities = self.ner.extract(full_text)
            
            self.corpus_entities.append({
                'news_id': news.get('id', len(self.corpus_entities)),
                'data': news.get('data'),
                'titolo': news.get('titolo'),
                'entities': entities
            })
    
    def get_top_entities(self, n: int = 10, entity_type: str = None) -> pd.DataFrame:
        """Restituisce le entità più frequenti."""
        all_entities = []
        
        for doc in self.corpus_entities:
            for e in doc['entities']:
                if entity_type is None or e.type == entity_type:
                    all_entities.append(e.text.lower())
        
        counter = Counter(all_entities)
        top = counter.most_common(n)
        
        return pd.DataFrame(top, columns=['Entità', 'Frequenza'])
    
    def get_entity_summary(self) -> pd.DataFrame:
        """Riassunto per tipo di entità."""
        type_counts = Counter()
        
        for doc in self.corpus_entities:
            for e in doc['entities']:
                type_counts[e.type] += 1
        
        df = pd.DataFrame(type_counts.items(), columns=['Tipo', 'Conteggio'])
        return df.sort_values('Conteggio', ascending=False)
    
    def find_cooccurrences(self, entity_type1: str, entity_type2: str) -> List[Tuple]:
        """Trova entità che appaiono insieme nello stesso documento."""
        cooccurrences = []
        
        for doc in self.corpus_entities:
            ent1_list = [e for e in doc['entities'] if e.type == entity_type1]
            ent2_list = [e for e in doc['entities'] if e.type == entity_type2]
            
            for e1 in ent1_list:
                for e2 in ent2_list:
                    cooccurrences.append((e1.text, e2.text))
        
        return Counter(cooccurrences).most_common(10)

# Dataset di notizie simulate
notizie = [
    {
        'id': 1,
        'titolo': 'Apple annuncia nuovo iPhone a Milano',
        'testo': 'Il CEO Tim Cook ha presentato l\'iPhone 15 al €1000. '
                 'L\'evento si è tenuto il 25/09/2024.',
        'data': '2024-09-25'
    },
    {
        'id': 2,
        'titolo': 'Ferrari trionfa a Monza',
        'testo': 'La Ferrari ha vinto il Gran Premio d\'Italia. '
                 'Charles Leclerc festeggia a Milano.',
        'data': '2024-09-01'
    },
    {
        'id': 3,
        'titolo': 'Unicredit e Intesa: nuova collaborazione',
        'testo': 'Le due banche italiane hanno firmato un accordo a Roma. '
                 'L\'investimento supera €500 milioni.',
        'data': '2024-10-15'
    },
    {
        'id': 4,
        'titolo': 'Google apre nuovo hub a Torino',
        'testo': 'Google investe in Italia con un centro ricerca. '
                 'Previste 200 assunzioni entro il 2025.',
        'data': '2024-10-20'
    },
]

# Analisi
analyzer = NewsAnalyzer()
analyzer.process_news(notizie)

print("="*60)
print("ESERCIZIO 3: NEWS ANALYZER")
print("="*60)

# Summary per tipo
print("\n📊 ENTITÀ PER TIPO:")
print("-" * 40)
print(analyzer.get_entity_summary().to_string(index=False))

# Top entità
print(f"\n📈 TOP 10 ENTITÀ (tutti i tipi):")
print("-" * 40)
print(analyzer.get_top_entities(10).to_string(index=False))

# Top locations
print(f"\n📍 TOP LOCATIONS:")
print("-" * 40)
print(analyzer.get_top_entities(5, 'LOC').to_string(index=False))

# Top organizations
print(f"\n🏢 TOP ORGANIZATIONS:")
print("-" * 40)
print(analyzer.get_top_entities(5, 'ORG').to_string(index=False))

# Dettaglio per notizia
print(f"\n{'='*60}")
print("📰 DETTAGLIO PER NOTIZIA")
print("="*60)
for doc in analyzer.corpus_entities:
    print(f"\n[{doc['data']}] {doc['titolo']}")
    for e in doc['entities']:
        print(f"  └─ [{e.type}] {e.text}")

---

# 5. Conclusione Operativa

## Cosa Abbiamo Imparato

| Concetto | Definizione | Applicazione |
|----------|-------------|--------------|
| **NER** | Identificazione entità nominate | Information Extraction |
| **BIO Tagging** | Schema di annotazione sequenziale | Training modelli NER |
| **Gazetteers** | Dizionari di entità note | Lookup veloce |
| **Rule-based NER** | Pattern regex per entità | Entità strutturate |
| **Feature Engineering** | Caratteristiche token | NER ML-based |

## Quando Usare Quale Approccio

| Tipo Entità | Approccio Consigliato | Perché |
|-------------|----------------------|--------|
| Date, email, numeri | **Regex** | Pattern fissi e prevedibili |
| Nomi aziende note | **Gazetteer** | Lista finita e stabile |
| Nomi persone generici | **ML/DL** | Variabilità alta |
| Dominio specifico | **Fine-tuning** | Lessico specializzato |

## Workflow Operativo

```python
# Template NER production

# 1. Per entità strutturate → Regex
import re
emails = re.findall(r'[\w.+-]+@[\w-]+\.[\w.-]+', text)

# 2. Per entità note → Lookup
known_orgs = {'apple', 'google', 'microsoft'}
found = [w for w in tokens if w.lower() in known_orgs]

# 3. Per NER completo → Librerie
# spaCy (locale, veloce)
import spacy
nlp = spacy.load("it_core_news_lg")
doc = nlp(text)
entities = [(ent.text, ent.label_) for ent in doc.ents]

# HuggingFace (cloud, accurato)
from transformers import pipeline
ner = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english")
```

## Errori Comuni da Evitare

1. **Regex troppo greedy**: pattern che matchano troppo
2. **Case sensitivity**: dimenticare `.lower()` nei lookup
3. **Overlap non gestito**: entità sovrapposte non risolte
4. **Valutazione token-level**: usare entity-level per NER
5. **Ignorare contesto**: guardare solo il token, non i vicini

---

# 6. Bignami — Scheda di Riferimento Rapido

## Tipi di Entità Standard

| Sigla | Significato | Esempi |
|-------|-------------|--------|
| PER | Persona | Mario Rossi, Angela Merkel |
| ORG | Organizzazione | Apple, ONU, Juventus |
| LOC | Luogo | Milano, Monte Bianco |
| GPE | Entità geopolitica | Italia, Roma (comune) |
| DATE | Data | 25/12/2024, lunedì |
| MONEY | Importo | €100, 1 milione |
| PERCENT | Percentuale | 5%, dieci per cento |

## Schema BIO

```
B-XXX = Begin: primo token entità tipo XXX
I-XXX = Inside: token successivi stessa entità
O     = Outside: non è un'entità
```

## Pattern Regex Comuni

```python
# Email
r'\b[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}\b'

# Telefono italiano
r'(?:\+39\s?)?(?:0\d{1,4}|\d{3})[\s.-]?\d{6,7}'

# Codice fiscale
r'\b[A-Z]{6}\d{2}[A-Z]\d{2}[A-Z]\d{3}[A-Z]\b'

# Data italiana
r'\b\d{1,2}[/-]\d{1,2}[/-]\d{2,4}\b'

# Importo euro
r'€\s?\d+(?:[.,]\d{2})?|
\d+(?:[.,]\d{2})?\s?€'
```

## Metriche NER

```python
# Entity-level evaluation
Precision = TP / (TP + FP)  # Dei predetti, quanti corretti
Recall = TP / (TP + FN)     # Dei gold, quanti trovati
F1 = 2 * P * R / (P + R)    # Media armonica
```

## Checklist Pre-Deploy

```
□ Pattern testati su edge cases
□ Gazetteers aggiornati
□ Overlap resolution implementato
□ Valutazione entity-level (non token)
□ Test su dati reali del dominio
□ Performance accettabile (>80% F1)
```

---
*Fine Lezione 33 — Named Entity Recognition (NER)*